# IRRmodel\*WCM calibration with PSO on KGE of $\sigma^0$

The v9 code is a final, cleaned and commented version.
This version, wrt to v8, implements an hourly ET0 calculated with hourly FAO-56 Penman-Monteith and a SWB model that is properly corrected for hourly calculation of the evapotranspiration.

# Dependencies

In [1]:
import sys
sys.path.append('../')

from modules.funcs import *
from modules.funcs_pso import *
from modules.funcs_plot import *
from modules.IRRI_WCM_model import *
from modules.EPOT_Hargreaves_pyeto import *

from scipy.signal import savgol_filter

# Data pre-processing

Input data formatting convention:
- ausiliary variables for extraction of data (directory name, file name, etc...)
- extraction into pd dataframe
- cleaning, resampling: drop unnecessary columns, set index to daily or hourly DateIndex

In [2]:
print('Starting...\n'
      +'#-------------------------------------------------------------\n'
      #+'Use of satellite-derived SM is provided for comparison, not calibration.\n'
     )
verbose = True if input("Verbose data extraction? (Describe datasets/files) [y/n]")=='y' else False

params = []; norma = ''

print('\n#-------------SWB model parameters-------------')
irri = True if input('Do you want to estimate irrigation as well as soil moisture? [y/n]')=='y' else False
year = int(input('Do you want to calibrate on year 2017 or 2020? [2017/2020]'))
freq = input('Daily or hourly frequency? [d/h]')

print('\n#-------------WCM model parameters-------------')
units = input('Calibrate WCM in linear or dB scale (determines scale of A)? [lin/db]')
opt_veg = 'NDVI'

print('\n#------------Optimization parameters-----------')
opt_calib = 'sigma0'
opt_cost = 'KGE'

PAR_str_add = input('Any addition to parameters\' names? [Type string to add]')
nrun = int(input('Number of runs? (10 is min to study distribution of parameters.) '))
n_particles = int(input('Number of particles: '))
n_step = int(input('Number of optimization steps: '))
optim = input('Global or Local PSO optimizer? [[global]/local] ')
if optim=='local': norma = 1 if input('Which norm? [l1/l2] ')=='l1' else 2
verbose_calib = True if input('Verbose during calibration? [y/n]')=='y' else False
automate = True if input('Run and save everything automatically? [y/n]')=='y' else False

Starting...
#-------------------------------------------------------------



Verbose data extraction? (Describe datasets/files) [y/n] n



#-------------SWB model parameters-------------


KeyboardInterrupt: Interrupted by user

## IRRmodel

In [ ]:
#----------------------------------------------------------------------------
# Budrio field data from platinum_df tables

# Freq: H
# Data extracted:
# - SWC (as input SWB/comparison)
# - rain (as input SWB)
# - irrigation (as input SWB)
# - temperature (as input SWB, ET0 estimate)

platinum_df = pd.ExcelFile('..\Inputs\Platinum_Budrio.xlsx', engine='openpyxl')
if year==2017: sheet='2017_1h'
if year==2020: sheet='2020_1h'
platinum_df = platinum_df.parse(sheet)
# platinum_df = pd.concat([platinum_df.parse('2017_1h'), platinum_df.parse('2020_1h')])

# Column 'Date' contains date+hour = hourly information
# Column 'Data' contains only date = daily information
platinum_df['Ora_1'] = pd.to_datetime(platinum_df['Ora'].astype('str')).apply(lambda x: x.time())
platinum_df['Data_1'] = pd.to_datetime(platinum_df['Data'].astype('str')).apply(lambda x: x.date())
platinum_df['Datetime'] = platinum_df.apply(lambda r : dtt.datetime.combine(r['Data_1'],r['Ora_1']),1)
platinum_df = platinum_df.drop(['ID', 'Data', 'Ora', 'Data_1', 'Ora_1', '214Pb[cps]'],axis=1)
platinum_df = platinum_df.set_index('Datetime')
platinum_resampled = platinum_df.resample('D', origin='end_day')
if verbose: platinum_df.info()

In [ ]:
#----------------------------------------------------------------------------
# Budrio field data from meteo tables

# Freq: H
# Data extracted:
# none
# Needed for eto from FAO PM 

meteo_df = pd.ExcelFile('..\Inputs\Budrio_Meteo.xlsx', engine='openpyxl')
meteo_df = meteo_df.parse(str(year))
# meteo_df = pd.concat([meteo_df.parse('2017'), meteo_df.parse('2020')]).set_index('ID')

# Column 'Date' contains date+hour = hourly information
meteo_df['Datetime'] = meteo_df.apply(lambda r : dtt.datetime.combine(r['Data'],r['Ora']),1)
meteo_df = meteo_df.set_index('Datetime')
meteo_df = meteo_df.drop(['Data', 'Ora'],axis=1)

if verbose: meteo_df.info()
# meteo_df.head()

In [ ]:
# Budrio database
# Freq: H
# Merging of Platinum+Meteo

meteo_h = pd.merge(right=platinum_df, left=meteo_df, on='Datetime')
if verbose: meteo_h.info()

In [ ]:
meteo_df.columns

In [ ]:
lat_deg = 44.570842547510622 # latitude of Budrio (deg)
temp_min = platinum_resampled.min()['Temperatura[°C]'].values
temp_max = platinum_resampled.max()['Temperatura[°C]'].values
temp_mean = platinum_resampled.mean()['Temperatura[°C]'].values
dates = platinum_resampled.asfreq().index
eto = timeseries( dates,
                 [ hargre(lat_deg, dates[i] , temp_min[i], temp_max[i], temp_mean[i])
                  for i in range(len(dates)) ] )
eto_df = pd.DataFrame(eto).rename(columns={0:'Date',1:'EPOT'}).set_index('Date')
if verbose: eto_df.info()

In [ ]:
# Computation of hourly eto by hourly FAO Penman-Monteith equation
# FAO-56 pag. 74

t = [x for x in meteo_h.index if x.year==year]

# Inputs
# ------
RH = meteo_h.loc[t]['Umidità aria Netsens [%] '].values if year==2017 else meteo_h.loc[t]['Umidità aria [%] '].values # mean hourly relative humidity [%]
T = meteo_h.loc[t]['Temperatura[°C]'].values # mean hourly air temperature [°C]
sol_rad = meteo_h.loc[t]['Radiazione solare [W/m2]  '].values # measured solar radiation [J m^-2 s^-1]
sol_rad_hour = sol_rad*3600*1e-6 # [MJ m^-2 h^-1]
T_dew = meteo_h.loc[t]['Punto di rugiada [C°] '].values # dew point [°C]
atmos_pres = meteo_h.loc[t]['Pressione atmosferica [hPa]  '].values/10 # atmospheric pressure [kPa]
wind_s = meteo_h.loc[t]['Velocità del vento [m/s]'].values
epot_hourly = [0.]*len(t)

# Params
# ------
for i in range(len(t)):
    
    lat_deg = 44.570842547510622 # latitude of Budrio (deg)
    lat_rad = np.deg2rad(lat_deg)
    long_deg = 11.53267200001359 # longitude of Budrio (deg)
    long_rad = np.deg2rad(long_deg)
    altitude = 11. # elevation [m]
    doy = t[i].dayofyear
    hour = t[i].hour
    mid_hour = hour+0.5
    
    # Extraterrestrial radiation (et_rad)
    Gsc = 0.0820 # MJ m^-2 min^-1
    dr = inv_rel_dist_earth_sun(doy) # inverse relative distance Earth-Sun
    d = sol_dec(doy) # solar declination [rad]
    phi = lat_rad # latitude [rad]
    lz = 360-15 # longitude of centre of local time zone (CET) (deg W of Greenwhich)
    lm = 360-long_deg # longitude of site (deg W of Greenwhich)
    b = 2*np.pi*(doy-81)/364
    Sc = 0.1645*np.sin(2*b)-0.1255*np.cos(b)-0.025*np.sin(b) # seasonal correction for solar time
    w = np.pi/12*((mid_hour+0.006667*(lz-lm)+Sc)-12) # solar time angle at midpoint of the period
    w1 = w-np.pi/24 # solar time angle at beginning [rad]
    w2 = w+np.pi/24 # solar time angle at end [rad]

    
    et_rad = 12*60/np.pi*Gsc*dr*((w2-w1)*np.sin(phi)*np.sin(d)+np.cos(phi)*np.cos(d)*(np.sin(w2)-np.sin(w1)))
    
    ws = np.arccos(-np.tan(lat_rad)*np.tan(d))
    N = 24/np.pi*ws # daylight hours
    Rs = sol_rad_hour[i] # measured solar radiation [MJ m^-2 h^-1]
    Rso = cs_rad(altitude, et_rad) # clear sky radiation [MJ m^-2 h^-1]
                                 
    avp = avp_from_tdew(T_dew[i]) # actual vapour pressure
    ni_rad = net_in_sol_rad(Rs) # net incoming solar radiation # use this formula, albedo is default value
                                 
    boltz_h = 2.043*1e-10 # Boltzmann constant [MJ m^-2 h^-1]
    e_thr = svp_from_t(T[i]) # saturation vapour pressure
    e_a = e_thr*RH[i]/100# average hourly actual vapour pressure
    Rs_ratio = Rs/Rso if Rs/Rso<1 else 1
    no_rad = boltz_h*T[i]**4*(0.34-0.14*np.sqrt(e_a))*(1.35*Rs_ratio-0.35) # net outgoing radiation corrected for hourly period
    R_n = ni_rad - no_rad # net radiation at grass surface
    
    u2 = wind_s[i]
    G = 0.1*R_n if hour in range(6,18,1) else 0.5*R_n # soil heat flux density, ref. eq. 45, 46 FAO56
    Delta = delta_svp(T[i]) #saturation slope
    gamma = psy_const(atmos_pres[i])# psychrometric constant
    
    epot_hourly[i] = (0.408*Delta*(R_n-G)+gamma*37/(T[i]+273)*u2*(e_thr-e_a))/(Delta+gamma*(1+0.34*u2))

In [ ]:
plt.plot(epot_hourly[:24*7])
plt.show()

## WCM

Frequency: multi-daily

Inputs:
- $\sigma^0$, $\theta$ (hourly, rounded to nearest midnight)
- vegetation descriptors (NDVI, LAI, CR) (average, daily)

In [ ]:
#----------------------------------------------------------------------------
# Sigma0 values

# Freq: D
# Daily values of backscattering from 2014 to 2022 (complete S1 series)
# Data extracted:
# - sigma0 values, VV and VH
# - angle of incidence of reference orbit (nearest to 40°)

sigma_df = pd.read_csv('..\Data\\budrio-half.csv', delimiter='\t');
sigma_df['Datetime'] = sigma_df.Date.apply(lambda x : pd.to_datetime(x))
angle_mean_95 = sigma_df.loc[sigma_df.Orb==95]['Angle[°]'].mean()
sigma_df['Angle[°]'] = sigma_df['Angle[°]'].apply(lambda x : angle_mean_95)
sigma_df.Date = sigma_df.Date.apply(lambda x : pd.to_datetime(x).round(freq='D'))
sigma_df = sigma_df.set_index('Date')
if verbose: sigma_df.info()

In [ ]:
#----------------------------------------------------------------------------
# NDVI values

# Freq: D
# Daily values of NDVI from 2017-4-24 to 2023-1-13 (complete S2 series)
# Data extracted:
# - NDVI (daily, interpolated)

ndvi_df = pd.read_csv('..\Data\\budrio-half_NDVI_discrete.csv', delimiter='\t');
ndvi_df.Date = ndvi_df.Date.apply(lambda x : pd.to_datetime(x))
ndvi_df = ndvi_df.set_index('Date')
ndvi_df = ndvi_df.resample('H').asfreq()
ndvi_df['NDVI_interp'] = ndvi_df.NDVI.interpolate(method='linear')
if verbose: ndvi_df.info()

# Input data

In [ ]:
# Input SWB
# Frequency: daily
# Data:
# - rain (sum, end of day) from platinum_resampled
# - eto (daily) from eto_df

if freq=='d':
    input_swb = pd.merge(right=eto_df, left=platinum_resampled.sum(),
                     right_on='Date', left_on='Datetime', right_index=True)\
                    .drop(['SWC[m3/m3]','Temperatura[°C]'], axis=1)
    input_swb['SWC[m3/m3]'] = platinum_resampled.mean()['SWC[m3/m3]']
    if verbose: input_swb.info()

# Input SWB
# Frequency: hourly
# Data:
# - rain (hourly) from platinum_df, linear interpolation
# - irrigation (hourly) from platinum_df
# - eto (daily) from eto_df, up-resampled hourly with linear interpolation

elif freq=='h':
    # eto_df_h = eto_df.resample('H').asfreq().interpolate(method='linear')
    eto_df_h = pd.DataFrame(timeseries(t, epot_hourly)).rename(columns={0:'Datetime', 1:'EPOT'})
    eto_df_h.set_index('Datetime')
    input_swb = pd.merge(right=eto_df_h, left=platinum_df,
                         on='Datetime',)
    input_swb.set_index('Datetime', inplace=True)
    input_swb['Pioggia[mm]'].interpolate(method='linear',inplace=True)
    input_swb['EPOT'].interpolate(method='linear',inplace=True)
    if verbose: input_swb.info()

In [ ]:
# Input WCM
# Frequency: daily
# Data:
# - sigma0 (multi-daily, rounded at nearest midnight)
# - NDVI (daily, interpolated)
# Merging is performed without any time offset, considering that the change
# in ndvi values anywhere is very small from one day to another 

input_wcm_sat = pd.merge(right=sigma_df, left=ndvi_df, on='Date', how='right')
input_wcm_sat.info()

In [ ]:
input_wcm = pd.merge(right=input_wcm_sat, left=platinum_df, on='Datetime',
                     how='inner')\
            .drop(['Pioggia[mm]','Irrigazione[mm]','Temperatura[°C]',
                   'Geometry_x'], axis=1)\
            .rename(columns={'SWC[m3/m3]':'SWC_h', 'Geometry_y':'Geometry',
                             'VV_norm[dB]':'VV', 'VH_norm[dB]':'VH', 'Angle[°]':'Angle'})

#----------------------------------------------------------------------------
# Convenience method for printing labels of columns that are interpolated
# only on the basis of their length being less than the length of the df,
# and not taking into consideration if they are being interpolated or extr

# print('The following columns will be interpolated: '
#       +f'{[x for x in input_wcm if len(input_wcm[x].dropna())<len(input_wcm)]}\n')
# if input('Proceed? [y/n]')!='y':
#      raise NameError('Stop and stare! I think I\'m going but I don\'t know where!')
#----------------------------------------------------------------------------

print(
    '\nNDVI values are limited below at the 24 of april 2017, so they are'+
    'backward propagated for 30 days in order to cover the whole Budrio period.'+
    ' There is a gap in SWC values: this gap is not interpolated but is used to'+
    'eliminate those days with nan SM values from the days of passage of the'+
    'satellite.\n'
)
input_wcm.NDVI_interp.interpolate(method='bfill',
                           limit_direction='backward',
                           limit=30,
                           # limit_area='inside', # THIS WORKS VERY BADLY IN EVERY WAY
                           inplace=True)

if freq=='d': input_wcm.index = input_wcm.Datetime.round('D')
elif freq=='h': input_wcm.index = input_wcm.Datetime
if verbose: input_wcm.info()

## Kc curves plots

In [ ]:
# Complete hours in 2017, 2020
t = [x for x in input_swb.index if x.year==year]
     
# Hours of passage of satellite, intersect with complete days
set1 = {x for x in input_swb.index}; set2 = {x for x in input_wcm.index};
intersect = np.sort(np.array([*set1.intersection(set2)]))
t_sat = [x for x in intersect if x.year==year]

# Hours of passage of S-2
set3 = {x for x in ndvi_df.index if x.year==year}; set4 = {*t}
intersect = np.sort(np.array([*set3.intersection(set4)]))
t_veg = [x for x in intersect]

In [ ]:
from modules.Kc_curve import *
Kc_array= Kc_curve(year, t)

In [ ]:
# 1x1 plot of inputs

if automate: opt_save = True
else: opt_save = True if input('Save plot? [y/n]')=='y' else False

fig, ax = plt.subplots(1, 1,constrained_layout=False,figsize=(12, 3), sharex=True,dpi=300,)
filename = f'kc_tomato' if year==2017 else f'kc_maize'
title=f''
times = t

veg       = ndvi_df.loc[t_veg]['NDVI_interp'].values # [-]
veg_point = ndvi_df.loc[t_veg]['NDVI'].values # [-]

#----------------------------------------------------------------------------
# Plot of first input

obs = Kc_array; obs_label='Kc [-]'
input0 = veg; input0_label='NDVI [-]'
input1 = veg_point
labely = f'{obs_label}'

plot_dict={
    'color':'k',
    'linestyle':'-',
    'linewidth':3,
    'marker':'',
}
ax.set_xlim(xmin=times[0], xmax=times[len(times)-1])
ax.plot(times, obs, label=obs_label, zorder=2, **plot_dict)
ax.legend(loc='upper left')
ax.set_ylabel(labely)

plot_dict={
    'color':'tab:green',
    'linestyle':'-',
    'linewidth':2,
    'marker':'',
}
ax0 = ax.twinx()
ax0.plot(t_veg, input0, label=input0_label, **plot_dict)
ax0.plot(t_veg, input1, marker='o', linestyle='-', color='tab:green')
ax0.legend(loc='upper right')
ax0.set_ylabel(input0_label)


#----------------------------------------------------------------------------
# Plot of inputs P, IRR, veg

# obs = P; obs_label='Rain'
# sim = IRR_obs; sim_label='Irri'
# input2 = ET; input2_label='ET'
# input4 = EPOT; input4_label='EPOT'
# labely = f'{obs_label}, {sim_label} [mm/h]'
# times = t
# 
# ax[3].bar(times, P, color='tab:gray', label=r'Rain')
# ax[3].bar(times, IRR_obs, color='tab:blue', label=r'Irri', zorder=2)
# ax[3].plot(times, input4, label=input4_label, color='tab:green', alpha=.5)
# ax[3].legend(loc='upper left')
# ax[3].set_ylabel(labely)
# 
# ax2 = ax[3].twinx()
# ax2.plot(times, input2, label=input2_label, color='tab:green', alpha=.8)
# ax2.legend(loc='upper right')
# ax2.set_ylabel(f'{input2_label} [mm/h]')

name=''
if opt_save:
    optim_choice = 'glo' if (optim=='')or(optim=='global') else 'local'
    name = filename # +f'_{n_particles}_{n_step}_{optim_choice}_{norma}'
    plt.savefig('Plot\\'+name+'.png')

# plt.close()

In [ ]:
# Choose subset of data for plotting

# List of 2017 and 2020 dates

# Complete hours in 2017, 2020
t = [x for x in input_swb.index if x.year==year]
     
# Hours of passage of satellite, intersect with complete days
set1 = {x for x in input_swb.index}; set2 = {x for x in input_wcm.index};
intersect = np.sort(np.array([*set1.intersection(set2)]))
t_sat = [x for x in intersect if x.year==year]

start_t = 0; end_t = start_t+7*24;
t = t[start_t:end_t]

In [ ]:
SWIR    = meteo_df.loc[t]['Radiazione solare [W/m2]  '].values
u2      = meteo_df.loc[t]['Velocità del vento [m/s]'].values
RH      = meteo_df.loc[t]['Umidità aria Netsens [%] '].values
P       = input_swb.loc[t]['Pioggia[mm]'].values # [mm]
IRR_obs = input_swb.loc[t]['Irrigazione[mm]'].values # [mm]
EPOT    = input_swb.loc[t]['EPOT'].values # [mm/day]
WW_obs  = input_swb.loc[t]['SWC[m3/m3]'].values # daily mean or hourly, with gaps
Thr     = input_swb.loc[t]['Temperatura[°C]'].values # mean hourly temperature [°C]
WW_sat  = input_wcm.loc[t_sat]['SWC_h'].values # punctual at hour (observed) [m3/m3]
veg     = input_wcm.loc[t_sat]['NDVI'].values # [-]
angle   = input_wcm.loc[t_sat]['Angle'].values # [°]
VV      = input_wcm.loc[t_sat]['VV'].values # [dB]
VH      = input_wcm.loc[t_sat]['VH'].values # [dB]
CR      = input_wcm.loc[t_sat]['CR'].values # [-]

## Plot inputs

In [ ]:
%matplotlib inline

import matplotlib as mplt
from matplotlib import pyplot as plt
plt.rcParams['font.size'] = 30
plt.rcParams.update({'font.size': 30})

In [ ]:
# 3x1 plot of inputs

if automate: opt_save = True
else: opt_save = True if input('Save triple plot with components? [y/n]')=='y' else False

fig, ax = plt.subplots(3, 1,constrained_layout=True,figsize=(9, 7), sharex=True,dpi=300,)
filename = f'inputs_et0'
title=f''
times = t
legend_dict = {
    #'fontsize':'large',
    'framealpha':1,
}

#----------------------------------------------------------------------------
# Plot of first input


obs = P; obs_label='Rain [mm]'
sim = IRR_obs; sim_label='Irrigation [mm]'
input0 = EPOT; input0_label='ET0 [mm/h]'
labely = f'Rain, irrigation [mm]'
times = t

plot_dict={
    'linestyle':'-',
    'linewidth':2,
    'marker':'',
}
ax0 = ax[0].twinx()
ax0.plot(times, obs, color='tab:gray', label=obs_label, **plot_dict)
ax0.plot(times, sim, color='tab:blue', label=sim_label, zorder=2, **plot_dict)
ax0.legend(loc='upper right', **legend_dict)
ax0.set_ylabel(labely)
ax0.set_ylim(0, 0.5)

plot_dict={
    'color':'k',
    'linestyle':'-',
    'linewidth':2,
    'marker':'',
}
ax[0].set_xlim(xmin=times[0], xmax=times[len(times)-1])
ax[0].plot(times, input0, label=input0_label, **plot_dict)
ax[0].legend(loc='upper left')
ax[0].set_ylabel(input0_label)
ax[0].set_ylim(0, 0.7)

#ax[0].set_title(title)

#----------------------------------------------------------------------------
# Plot of second inputs

obs = Thr; obs_label=r'T [°C]'
input3 = SWIR; input3_label=r'SWIR [W/$m^2$]'

plot_dict={
    'color':'tab:orange',
    'linestyle':'-',
    'linewidth':2,
    'marker':'',
}

ax[1].plot(times, input3, label=input3_label, **plot_dict)
ax[1].set_ylabel(input3_label)
ax[1].legend(loc='upper left')
ax[1].set_ylim(0, 1100)


plot_dict={
    'color':'tab:blue',
    'linestyle':'-',
    'linewidth':2,
    'marker':'',
}

ax3 = ax[1].twinx()
ax3.plot(times, obs, label=obs_label, zorder=-10, **plot_dict)
ax3.legend(loc='upper right')
ax3.set_ylabel(obs_label)
ax3.set_ylim(0, 35)

#----------------------------------------------------------------------------
# Plot of third inputs

obs = RH; obs_label='Rel. air humidity [%]'
input1 = u2; input1_label='Wind speed [m/s]'
labely = obs_label

plot_dict={
    'color':'tab:cyan',
    'linestyle':'-',
    'linewidth':2,
    'marker':'',
}

ax[2].set_xlim(xmin=times[0], xmax=times[-1])
ax[2].plot(times, obs, label=obs_label, alpha=.8, **plot_dict)
ax[2].set_ylabel(labely)
ax[2].set_ylim(0, 120)

plot_dict={
    'color':'tab:grey',
    'linestyle':'-',
    'linewidth':2,
    'marker':'',
}

ax1 = ax[2].twinx()
ax1.plot(times, input1, label=input1_label, **plot_dict)

ax[2].legend(loc='upper left')
ax1.legend(loc='upper right', framealpha=1)
ax1.set_ylabel(input1_label)
ax1.set_ylim(0,7)

#----------------------------------------------------------------------------
from matplotlib.ticker import AutoMinorLocator


for asse in [*ax,ax0,ax1,ax3]: 
    # Text in the x-axis will be displayed in 'YYYY-mm' format.
    asse.minorticks_on()
    asse.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d, %H'))
    asse.xaxis.set_minor_locator(AutoMinorLocator())
    # Rotates and right-aligns the x labels so they don't crowd each other.
    for label in asse.get_xticklabels(which='major'):
        label.set(rotation=30, horizontalalignment='right')
    # asse.xaxis.label.set_fontsize(18)
    # asse.yaxis.label.set_fontsize(18)
    # map(lambda p: p.set_fontsize(18), asse.get_xticklabels())
    # map(lambda p: p.set_fontsize(18), asse.get_yticklabels())

name=''
if opt_save:
    optim_choice = 'glo' if (optim=='')or(optim=='global') else 'local'
    name = filename # +f'_{n_particles}_{n_step}_{optim_choice}_{norma}'
    plt.savefig('Plot\\'+name+'.png')

# plt.close()

In [ ]:
raise NameError('Stop right here!')

# Calibration SWB+WCM

In [ ]:
thr_R = 0.75

In [ ]:
def pso_calib_irri(PAR):
    """Ausiliary function for PSO optimization"""
    global inputs
    global user_in
    n_particles = PAR.shape[0]
    err = np.zeros(n_particles)
    for i in range(n_particles):
        WW,IRR,sig0,KGE = IRR_WCM(PAR[i], inputs, user_in)
        err[i] = 1 - KGE
    return err

In [ ]:
# Calibration SWB+WCM

print('Starting calibration...\n'+
      '#-------------------------------------------------------------\n')

A=0.3; B=1.5; C=-15; D=30 # guess params for WCM
W_max=    100;   # [mm] water content, maximum (not normalized)
WW_fc=    0.32; # [m3/m3] water content at field capacity
WW_w=     0.09; # [m3/m3] water content at wilting point
rho_st=   0.4;  # [-] crop specific depletion fraction
Kc0=      1/2;  # [-] crop specific coefficient
# if freq=='h': rho_st/=24; Kc0/=1

# PAR default
PAR_str = ['A', 'B', 'C', 'D', 'W_max', 'WW_fc', 'WW_w', 'rho_st', 'Kc0']
if units=='db': PAR_str = [el + '_db' for el in PAR_str]

# PAR to calibrate
# A, B, C, D, W_max, WW_fc, WW_w, rho_st, Kc0
PAR     = [A, B, C, D, W_max, WW_fc, WW_w, rho_st, Kc0]
PARn_str= ['A', 'B', 'C', 'D', 'W_max', 'WW_fc', 'WW_w', 'rho_st', 'Kc0']
inputs  = [t, t_sat, P, IRR_obs, EPOT, Kc_array, WW_obs, WW_sat, veg, angle, VV]
user_in = [irri, units]

if units=='lin':
    bounds = (
        np.array([0.01,0.1,-30, 10,   5, 0.29, 0.06,0.25, 0.35]), # low
        np.array([1,   3,   -5,100,  30, 0.35, 0.12,0.50, 0.65]),  # up
    )
elif units=='db':
    bounds = (
        np.array([ -20,0.1,-30, 10,   1, 0.3, 0.01,]), # low
        np.array([ -10,3,   -5,100, 150, 0.5, 0.2, ]),  # up
    )
#     bounds = (
#         np.array([0.01,0.1,-30, 10,   1, 0.3, 0.01,0.01, 0.001]), # low
#         np.array([1,   3,   -5,100, 150, 0.5, 0.2, 0.8,  1]),  # up
#     )
else: raise NameError(f'Scale can only be lin or db but {scale} was provided.')

#-----------------------------------------------------------------------------
from pyswarms.backend.handlers import OptionsHandler

params = []
start = time.time()
for i in range(int(nrun)):
    
    print('Run number ', i+1)
    if (optim=='global')or(optim==''):
        optim='global'
        # options = {'c1': 0.5, 'c2': 0.9, 'w': 0.6}
        options = {'c1': 2.05, 'c2': 2.05, 'w': 0.6}
        oh_strategy = {"w":'exp_decay', 'c1':'lin_variation', 'c2':'lin_variation'}
        optimizer = ps.single.GlobalBestPSO(n_particles=int(n_particles),
                                            dimensions=len(PAR),
                                            options=options,
                                            bounds=bounds,
                                            oh_strategy=oh_strategy,
                                           )
    elif optim=='local':
        options = {'c1': 0.4, 'c2': 0.4, 'w': 0.8, 'k':int(0.1*n_particles), 'p':norma }
        optimizer = ps.single.LocalBestPSO(n_particles=n_particles,
                                           dimensions=len(PAR),
                                           options=options,
                                           bounds=bounds,
                                           oh_strategy={"w":'exp_decay',
                                                         'c1':'lin_variation',
                                                         'c2':'lin_variation'})
    else: raise NameError('Please provide an accepted option.')
    
    R=0.5*thr_R
    while R<thr_R:
        cost, PARn = optimizer.optimize(pso_calib_irri, n_step, verbose=verbose_calib)#, **PAR)
    
    params.append(PARn)
    end = time.time()
    if i==0:
        time_sec = round(end-start, 2)
        time_min =  round((end-start)/60, 2)
        print(time_min, ' min for last run,', time_min*nrun , ' min estimated')
    print('Time left: ', round((start+time_sec*nrun-end)/60,2), 'min')

timestr = time.strftime("%y%m%d-%H%M%S"); print('Timestring: ', timestr)

In [ ]:
print('Timing: ', round((end-start)/60/nrun, 2), ' min for 1 run,', round((end-start)/60, 2), 'min total')

# Parameters' study

In [ ]:
matrix = np.array(
    [
        np.array(
            [ params[i][j] for i in range(len(params)) ])
        for j in range(len(PAR))
    ]
)

PAR_dict = {
    'A':     [round(A, 2) ],
    'B':     [round(B, 2) ],
    'C':     [round(C, 2) ],
    'D':     [round(D, 2) ],
    'W_max': [round(W_max, 2) ],
    'WW_fc': [round(WW_fc, 2) ],
    'WW_w':  [round(WW_w, 2) ],
    'rho_st':[round(rho_st, 2) ],
    'Kc0' :  [round(Kc0, 2)]
}

PARn = []
PARn_dev = []

if automate: opt_save=True
else: opt_save = True if input('Save histograms of params? [y/n]')=='y' else False

for label in PAR_dict:
    if label in PARn_str:
        i = PARn_str.index(label)
        
        if len(params)==1:
            PARn=[x[0] for x in matrix]
            PARn_dev=[0 for x in matrix]
        else:
            hist_kwargs={'alpha':.5, 'label':f'param {label}'}
            fitline_kwargs={'linestyle':'-', 'label':f'fit_{label}'}
            data = matrix[i]
            nbins = 10 if len(params)<100 else np.sqrt(len(params))-4
            plt.xlim(bounds[0][i], bounds[1][i]);
            counts, bins, pads, popt, pcov = hist_gauss_fit(
                data, nbins=10, hist_kwargs=hist_kwargs, fitline_kwargs=fitline_kwargs,
                title=f'{label}', density=False,
                opt_save=opt_save, dir_name='Plot\\', opt_name=f'{timestr}_hist_{label}',
                func=gauss)
            PARn.append(popt[1])
            PARn_dev.append(popt[2])    
            plt.show()
        
        PAR_dict[label].append('cal')
        PAR_dict[label].append([round(bounds[0][i],2), round(bounds[1][i],2)])
        PAR_dict[label].append(np.round(PARn[i], 3))
        PAR_dict[label].append(np.round(PARn_dev[i], 3))
    else:
        PAR_dict[label].append('fix')
        PAR_dict[label].append(['/', '/'])
        PAR_dict[label].append(PAR_dict[label][0])
        PAR_dict[label].append('/')

PAR_dict

# Model output - plots sim VS obs for SM, sigma0

In [ ]:
# Model validation and output

if not automate:
    if input('Do you want to plot with user-defined parameters\' values ? [y/n]')=='y':

        # PAR  = [A, B, C, D, W_max, Kc]
        # PARn = [0.29, 3.36, -15.75, 40.92, 100.18, 0.58] # [lin]
        # PARn = [-15, 0.49, -21.06, 33.77, 100.18, 0.44] # [db] # NEED TO RUN AGAIN IF UNITS CHANGE
        # PARn = [0.29,2.14,-14.69,35.79,100,0.72] # [lin] new params Kc
        # PARn = [0.28, 0.71, -12.88, 22.78, 16.56, 0.50] # [lin] Wmax in [1,150]
        # PARn = [0.3179471,1.75074996,-18.0340179,41.18898174,100.05601716]
        # WW,IRR,sigma0,KGE = IRR_WCM(PARn, inputs, user_in)
        
        # # Model version with all params
        # PARn = [0.3, 1.4, -15, 40, 100, 0.32, 0.08, 0.4/24, 0.05]
        
        timestr = time.strftime("%y%m%d-%H%M%S"); print('Timestring: ', timestr)
        
        # PARn = [0.3, 3, -5, 40, 100, 0.32, 0.08, 0.4/24, 1/24]
        # PAR_tot = [PAR_dict[label][3] for label in PAR_dict]
        PAR_tot = [0.357, 0.808, -16.258, 32.728, 4.136, 0.339, 0.088, 0.072, 0.005]
        timestr = '230131-165510'
        print(PAR_tot)
        WW,IRR,sigma0,KGE = IRR_WCM_allpar(PAR_tot, inputs, user_in)
    else: WW,IRR,sigma0,KGE = IRR_WCM(PARn, inputs, user_in)

else: WW,IRR,sigma0,KGE = IRR_WCM(PARn, inputs, user_in)

In [ ]:
if automate: opt_save_table = True
else: opt_save_table = True if input('Save table with machine params? [y/n]')=='y' else False

fig, ax = plt.subplots(figsize=(14,5))
row_height=0.1; col_width=0.7

# Table with machine, inputs params
timing = 'daily' if freq=='d' else 'hourly'
cellText = [
    ['Units', units],
    # ['Cost function', opt_cost],
    # ['Calib on', opt_calib],
    ['Optimizer:', optim],
    ['Optimizer: options', options],
    ['Optimizer: strategy', oh_strategy],
    ['N runs, particles, optim. steps', str(nrun)+', '+str(n_particles)+', '+str(n_step)],
    ['Time frequency', timing],
    ['Vegetation descriptor', opt_veg],
]
table = ax.table(cellText=cellText,
                     cellLoc='center', loc='center',
                     colLabels=None, rowLabels=None)

for (row, col), cell in table.get_celld().items():
    cell.set_height(row_height)
    cell._loc = 'center'

table.auto_set_font_size(False); table.set_fontsize(20)
table.auto_set_column_width(col=[x for x in range(len(PAR_dict))])
ax.axis('tight'); ax.axis('off')

if opt_save_table: plt.savefig(f'Plot\\{timestr}_table_mach.png', dpi=100)
plt.show()

In [ ]:
if automate: opt_save_table = True
else: opt_save_table = True if input('Save table with params? [y/n]')=='y' else False

fig, ax = plt.subplots(figsize=(24,5))
row_height=0.1; col_width=0.1

# Table with params
cellText = [[PAR_dict[label][i] for label in PAR_dict] for i in range(5)]
colLabels = [x for x in PAR_dict]
rowLabels = ['Guess/fix value', 'Cal/fix', 'Bounds', 'Mean', 'St.dev.']

#---------------------------------
table = ax.table(cellText=cellText,
                  cellLoc='center', loc='center',
                  colLabels=colLabels,
                  rowLabels=rowLabels,
                )

for (row, col), cell in table.get_celld().items():
    if row == 0: # or col == 0:
        cell.set_text_props(weight='bold')
    cell.set_height(row_height)
    cell._loc = 'center'

table.auto_set_font_size(False); table.set_fontsize(20)
table.auto_set_column_width(col=[x for x in range(len(PAR_dict))])
ax.axis('tight'); ax.axis('off')

if opt_save_table: plt.savefig(f'Plot\\{timestr}_table.png')
plt.show()

In [ ]:
if automate: opt_save=True
else: opt_save = True if input('Save log with parameters\' values? [y/n]')=='y' else False

if opt_save:
    for label in PAR_dict:
        i = PARn_str.index(label)
        PAR_dict[label].append(matrix[i])
        # PAR_dict['index'] = ['Guess/fix value', 'Cal/fix', 'Bounds', 'Mean', 'St.dev.','Values']
        # pd.DataFrame.from_dict(PAR_dict).set_index('index').to_csv(timestr+'_params.txt', sep=',')
        with open('Plot\\'+timestr+'_params.txt', 'w') as f: 
            for key, value in PAR_dict.items(): 
                f.write('%s:%s\n' % (key, value))

## Triple plot SM+$\sigma^0$+inputs

In [ ]:
#-----------------------------------------------------------------------
# Triple plot

if automate: opt_save = True
else: opt_save = True if input('Save triple plot SM+s0+inputs? [y/n]')=='y' else False

fig, ax = plt.subplots(3, 1,constrained_layout=False,figsize=(16, 9), sharex=True,dpi=300,)
filename = f'triple_'+units+'_'+PAR_str_add

#----------------------------------------------------------------------------
# Box with params

PAR_text='Parameters\n'
try:
    for i in range(len(PAR_str)): PAR_text+=f'    {PAR_str[i]} = {PAR_tot[i]}'
except NameError:
    for label in PAR_dict: PAR_text+=f'    {label} = {PAR_dict[label][3]}'

fig.text(0.5, 0.95,
         PAR_text, ha='center', va='center',
         bbox={'facecolor':'white', 'edgecolor':'black'},
         transform=fig.transFigure,
         fontsize='large',
        )


def bias(obs, sim):
    """distance between obs' and sim's mean values"""
    if len(obs)==len(sim):
        return np.mean(obs-sim)
    else: raise ValueError(
        f'obs and sim must have same first dimension, but have shapes {np.shape(obs)} and {np.shape(sim)}')
    
#-----------------------------------------------------------------------
from scipy.signal import savgol_filter

plot_triple(fig, ax, times1=t_sat, data1=[VV, sigma0], data1_label=r'$\sigma^0$[dB]', 
            input1=veg, input1_label=opt_veg,
            times2=t, data2=[WW_obs, WW], data2_label='SM [-]',
            input2=[], input2_label='',
            times3=t, data3=
            [P,
             IRR_obs,
             savgol_filter(EPOT, window_length=24, polyorder=1, mode='interp')],
            data3_label=['Rain [mm]', 'Irrigation [mm]', 'ET0 [mm/h]'],
            user_input=[irri],
           )

name=''
if opt_save:
    optim_choice = 'glo' if (optim=='')or(optim=='global') else 'local'
    name = timestr+filename # +f'_{n_particles}_{n_step}_{optim_choice}_{norma}'
    plt.savefig('Plot\\'+name+'.png')

plt.show()

## Scatterplot

In [ ]:
if automate: opt_save = True
else: opt_save = True if input('Save scatterplot of SM? [y/n]')=='y' else False

filename = f'scatter_'+'sm_'+units+PAR_str_add # 'SM'

plot_sim_vs_obs(sim=WW, obs=WW_obs, quantity='SM', um='[-]')
    
if opt_save: plt.savefig('Plot\\'+timestr+'_'+filename+'.png')

In [ ]:
if automate: opt_save = True
else: opt_save = True if input('Save scatterplot of SM? [y/n]')=='y' else False

filename = f'scatter_'+'sigma0_'+units+PAR_str_add # 'sigma0'

plot_sim_vs_obs(sim=sigma0, obs=VV, quantity=r'$\sigma^0$', um='[dB]')
    
if opt_save: plt.savefig('Plot\\'+timestr+'_'+filename+'.png')

# Components

In [ ]:
A, B, C, D, W_max, WW_fc, WW_w, rho_st, Kc0 = [PAR_dict[label][3] for label in PAR_dict]


W_fc   = WW_fc*W_max # field capacity [mm]
W_w    = WW_w*W_max # wilting point [mm]
ET     = np.array([.0]*len(t)) # evapotranspiration
Ks     = np.array([.0]*len(t)) # water stress coefficient
rho    = np.array([.0]*len(t)) # depletion fraction
PS     = np.array([.0]*len(t)) # deep percolation
W      = np.array([.0]*len(t)) # water content [mm]
W[0]   = WW_obs[0]*W_max # initial value of sm [mm]

if irri==True: IRR = [.0]*len(t) # water content
else: IRR = IRR_obs

for i in [i+1 for i in range(len(t)-1)]:
    
    DOY=t[i].dayofyear
    
    # Build Ks curve
    Kci = Kc_array[i]*Kc0
    
    # Compute depletion fraction
    if i<25:EPOT_day=np.sum(EPOT[i:i+24])
    else: EPOT_day=np.sum(EPOT[i-24:i])
    rho[i]=rho_st+0.04*(5/24-Kci*EPOT[i])
    
    if W[i-1]>=(1-rho[i])*W_fc:
        Ks[i]=1
    elif (W[i-1]>W_w)and(W[i-1]<(1-rho[i])*W_fc):
        Ks[i]=(W[i-1]-W_w)/((1-rho[i])*(W_fc-W_w))
    else: Ks[i]=0
    
    # Adjusted evapotranspiration
    ET[i] = EPOT[i]*Kci*Ks[i]
    
    # Irrigation estimate (for summer season only)
    # Irrigation is estimated as the amount of water needed from the day
    # before to take water content up to field capacity
    if irri==True:
        if np.logical_and(DOY>START,DOY<START+100): # summer season
            if W[i-1]<=(1-rho[i])*W_fc: IRR[i]=W_fc-W[i-1]
    
    
    # Water balance [mm]
    W[i]=W[i-1]+P[i]+IRR[i]-ET[i]
    
    # Computation of deep percolation (water above field capacity)
    if W[i]>W_fc:
        PS[i]=W[i]-W_fc
        W[i]=W_fc

In [ ]:
if automate: opt_save = True
else: opt_save = True if input('Save triple plot with components? [y/n]')=='y' else False

fig, ax = plt.subplots(4, 1,constrained_layout=False,figsize=(16, 9), sharex=True,dpi=300,)
filename = f'components_'+units+'_'+PAR_str_add

#----------------------------------------------------------------------------
# Box with params

PAR_text='Parameters\n'

try:
    for i in range(len(PAR_str)): PAR_text+=f'    {PAR_str[i]} = {PAR_tot[i]}'
except NameError:
    for label in PAR_dict: PAR_text+=f'    {label} = {PAR_dict[label][3]}'

fig.text(0.5, 0.95,
         PAR_text, ha='center', va='center',
         bbox={'facecolor':'white', 'edgecolor':'black'},
         transform=fig.transFigure,
         fontsize='large',
        )

#----------------------------------------------------------------------------
# Plot of sigma0 components

obs = (sig0s); obs_label=r'$\sigma^0_{soil}$'
sim = norm_fit(sig0v, min(sig0s), max(sig0s)); sim_label=r'$\sigma^0_{veg}$'
input0 = veg; input0_label='veg'
labely = r'$\sigma^0$[dB]'
times = t_sat
marker='o'; linestyle='-'

title=f'{sim_label}, {obs_label}, {input0_label}'

ax[0].set_xlim(xmin=times[0], xmax=times[len(times)-1])
ax[0].plot(times, sim, c='darkkhaki', label=sim_label,
           linestyle=linestyle, marker=marker, )#alpha=.4, zorder=-1)
ax[0].plot(times, obs, c='goldenrod', label=obs_label,
           linestyle=linestyle, marker=marker, zorder=-1)
ax[0].legend(loc='upper left')
ax[0].set_title(title)
ax[0].set_ylabel(labely)

ax0 = ax[0].twinx()
ax0.plot(times, input0, label=input0_label, color='tab:green')
ax0.legend(loc='upper right')
ax0.set_ylabel(input0_label+' [-]')

#----------------------------------------------------------------------------
# Plot of soil moisture and sigma0, simulations

obs = sig0; obs_label=r'$\sigma^0$'
input3 = W; input3_label='SM'
labely = r'$\sigma^0$[dB]'
marker='o'; linestyle='-'

ax[1].plot(t, input3, color='tab:blue', label=input3_label, zorder=-10)
ax[1].legend(loc='upper left')
ax[1].set_ylabel('SM [mm]')

ax3 = ax[1].twinx()
ax3.plot(t_sat, sig0, color='tab:orange', label=obs_label, marker=marker, linestyle=linestyle, zorder=10)
ax3.legend(loc='upper right')
ax3.set_ylabel(labely)

#----------------------------------------------------------------------------
# Plot of SWB components

obs = norm(Kc_array); obs_label='Kc'
sim = norm(Ks); sim_label='Ks'
input1 = norm(Kc_array); input1_label='Kc'
labely = '[-]'
times = t

if irri:
    IRRmatrix = np.array( [ [IRR[i], IRR_obs[i]] for i in range(len(IRR)) if not np.isnan(IRR_obs[i]) ] )
    R_IRR=np.corrcoef(IRRmatrix,rowvar=False)[0][1]; print('R_IRR (IRR vs IRR_obs)=', R_IRR)
    B_IRR=bias(np.array([e[0] for e in IRRmatrix]), np.array([e[1] for e in IRRmatrix]))
    irri_title = f'sumIRR_obs={np.sum(IRR_obs):.2f}, '+\
                 f'sumIRR_sim={np.sum(IRR):.2f}, '+\
                 f'R_IRR={R_IRR:.2f}, '+\
                 f'bias_IRR={B_IRR:.2f}, '
else: irri_title=''

title= ''#  f'SM_obs VS SM_sim - '+f'{irri_title}'+\
    # f'R_SM={R:.2f}, bias_SM={BIAS:.2f}'

ax[2].set_xlim(xmin=times[0], xmax=times[-1])
ax[2].plot(times, sim, c='teal', label=sim_label, linestyle='-', alpha=.8)
# ax[2].plot(times, obs, c='seagreen', label=obs_label, linestyle='-', lw=2)
ax[2].legend(loc='upper left')
ax[2].set_title(title)
ax[2].set_ylabel(labely)

ax1 = ax[2].twinx()
ax1.plot(times, input1, label=input1_label, color='tab:green')
ax1.legend(loc='upper right')
ax1.set_ylabel(input1_label+' [mm/h]')

#----------------------------------------------------------------------------
# Plot of inputs P, IRR, veg

obs = P; obs_label='Rain'
sim = IRR_obs; sim_label='Irri'
input2 = ET; input2_label='ET'
input4 = EPOT; input4_label='EPOT'
labely = f'{obs_label}, {sim_label} [mm/h]'
times = t

ax[3].bar(times, P, color='tab:gray', label=r'Rain')
ax[3].bar(times, IRR_obs, color='tab:blue', label=r'Irri', zorder=2)
ax[3].plot(times, input4, label=input4_label, color='tab:green', alpha=.5)
ax[3].legend(loc='upper left')
ax[3].set_ylabel(labely)

ax2 = ax[3].twinx()
ax2.plot(times, input2, label=input2_label, color='tab:green', alpha=.8)
ax2.legend(loc='upper right')
ax2.set_ylabel(f'{input2_label} [mm/h]')

name=''
if opt_save:
    optim_choice = 'glo' if (optim=='')or(optim=='global') else 'local'
    name = timestr+filename # +f'_{n_particles}_{n_step}_{optim_choice}_{norma}'
    plt.savefig('Plot\\'+name+'.png')

# plt.close()